# EMG Signal Processing Assignment

## Setup
    Import necessary libraries: numpy, scipy, matplotlib.

## Part 1: Signal Filtering

    Task: Implement a Butterworth filter to remove high-frequency noise from the EMG signal.
    Task: Compare the frequency response of the original and filtered signals using a frequency plot.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt, find_peaks

fs = 1000
t = np.arange(0, 1, 1/fs)
emg_signal = np.sin(2*np.pi*10*t) + 0.5*np.random.randn(len(t))

plt.figure(figsize=(10, 6))
plt.plot(t, emg_signal, label='Original EMG Signal')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.title('EMG Signal')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
T = 1
cutoff = 12
nyq = 0.5 * fs
order = 2
n = int(T*fs)

def butter_filter(data, cutoff, fs, order):
  normal_cutoff = cutoff/nyq
  b, a = butter(order, normal_cutoff, btype='low', analog=False)
  y = filtfilt(b, a, data)
  return y

y = butter_filter(emg_signal, cutoff, fs, order)

plt.plot(t, y, label='Filtered signal')
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
frequencies = np.fft.fftfreq(n, d=1/fs)
emg_signal_fft = np.fft.fft(emg_signal)
y_fft = np.fft.fft(y)


plt.figure(figsize=(10, 6))
plt.plot(frequencies[:n//2], np.abs(emg_signal_fft)[:n//2], label='Original EMG Signal')
plt.xlabel('Frequency')
plt.ylabel('Power')
plt.title('Frequency Response of Original EMG Signal')
plt.legend()
plt.grid(True)
plt.show()

plt.figure(figsize=(10, 6))
plt.plot(frequencies[:n//2], np.abs(y_fft)[:n//2], label='Filtered EMG Signal')
plt.xlabel('Frequency')
plt.ylabel('Power')
plt.title('Frequency Response of Filtered EMG Signal')
plt.legend()
plt.grid(True)
plt.show()


## Part 2: Peak Detection

    Task: Utilize scipy.signal.find_peaks to detect peaks in the filtered EMG signal.
    Question: What parameters can be adjusted in find_peaks function to improve peak detection accuracy?
    Task: Plot the original signal with markers indicating the detected peaks.

In [ ]:
from scipy.signal import find_peaks
peaks, properties = find_peaks(y, height=0, distance=20)

plt.figure(figsize=(10, 6))
plt.plot(t, y, label='Filtered EMG Signal')
plt.plot(t[peaks], y[peaks], "x", label='Detected Peaks')
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.title('Filtered EMG Signal with Detected Peaks')
plt.legend()
plt.grid(True)
plt.show()

Answer the Question here:

In [ ]:
#Parameters like height, threshold, width, distance can be adjusted to find peaks accurately.

## Part 3: Peak Start and End Time Detection

    Task: Develop a method to determine the start and end times of each detected peak.
    Question: Why is it necessary to identify the precise onset and offset times of muscle activation in EMG signals?
    Task: Visualize the original signal with annotated start and end times of peaks.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt, find_peaks

fs = 1000
t = np.arange(0, 1, 1/fs)
emg_signal = np.sin(2*np.pi*10*t) + 0.5*np.random.randn(len(t))

cutoff = 12
nyq = 0.5 * fs
order = 2
T=1

def butter_filter(data, cutoff, fs, order):
  normal_cutoff = cutoff/nyq
  b, a = butter(order, normal_cutoff, btype='low', analog=False)
  y = filtfilt(b, a, data)
  return y

y = butter_filter(emg_signal, cutoff, fs, order)

peaks, _ = find_peaks(y, height=0, distance=20)

######


plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.title('Filtered EMG Signal with Annotated Start and End Times of Peaks')
plt.legend()
plt.grid(True)
plt.show()


Answer the Question here:

In [ ]:
#

## Part 4: Envelope Extraction

    Task: Implement the Hilbert transform to extract the envelope of the filtered EMG signal.
    Question: What information does the envelope of an EMG signal provide about muscle activity?
    Task: Plot the original signal overlaid with its envelope.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt, find_peaks, hilbert

# Generate example EMG signal with noise
fs = 1000  # Sampling frequency (Hz)
t = np.arange(0, 1, 1/fs)  # Time vector (1 second)
emg_signal = np.sin(2*np.pi*10*t) + 0.5*np.random.randn(len(t))

cutoff = 12
nyq = 0.5 * fs
order = 2
T=1

def butter_filter(data, cutoff, fs, order):
  normal_cutoff = cutoff/nyq
  b, a = butter(order, normal_cutoff, btype='low', analog=False)
  y = filtfilt(b, a, data)
  return y

y = butter_filter(emg_signal, cutoff, fs, order)

analytic_signal = hilbert(y)
envelope = np.abs(analytic_signal)

plt.figure(figsize=(10, 6))
plt.plot(t, y, label='Filtered EMG Signal', color='orange')
plt.plot(t, envelope, label='Envelope', color='blue')
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.title('Filtered EMG Signal Overlaid with Envelope')
plt.legend()
plt.grid(True)
plt.show()


Answer the Question here:

In [ ]:
#

## Part 5: Time Domain Analysis

    Task: Calculate key metrics such as mean, variance, and RMS (Root Mean Square) of the cleaned EMG signal.
    Task: Interpret the calculated metrics in the context of muscle activity and signal characteristics.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt, hilbert

# Generate example EMG signal with noise
fs = 1000  # Sampling frequency (Hz)
t = np.arange(0, 1, 1/fs)  # Time vector (1 second)
emg_signal = np.sin(2*np.pi*10*t) + 0.5*np.random.randn(len(t))

cutoff = 12
nyq = 0.5 * fs
order = 2
T=1

def butter_filter(data, cutoff, fs, order):
  normal_cutoff = cutoff/nyq
  b, a = butter(order, normal_cutoff, btype='low', analog=False)
  y = filtfilt(b, a, data)
  return y

y = butter_filter(emg_signal, cutoff, fs, order)

mean_value = np.mean(y)
variance_value = np.var(y)
rms_value = np.sqrt(np.mean(y**2))

print(f"Mean: {mean_value:.4f}")
print(f"Variance: {variance_value:.4f}")
print(f"RMS: {rms_value:.4f}")

## Part 6: Frequency Domain Analysis

    Task: Apply the Fast Fourier Transform (FFT) to analyze the frequency components of the cleaned EMG signal.
    Task: Plot the frequency spectrum of the EMG signal and identify prominent frequency peaks.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt

# Generate example EMG signal with noise
fs = 1000  # Sampling frequency (Hz)
t = np.arange(0, 1, 1/fs)  # Time vector (1 second)
emg_signal = np.sin(2*np.pi*10*t) + 0.5*np.random.randn(len(t))
cutoff = 12
nyq = 0.5 * fs
order = 2
T=1

def butter_filter(data, cutoff, fs, order):
  normal_cutoff = cutoff/nyq
  b, a = butter(order, normal_cutoff, btype='low', analog=False)
  y = filtfilt(b, a, data)
  return y

y = butter_filter(emg_signal, cutoff, fs, order)
n = int(T*fs)
frequencies = np.fft.fftfreq(n, d=1/fs)
fft_values = np.fft.fft(y)

power = np.abs(fft_values) / n

positive_frequencies = frequencies[:n//2]
positive_power = power[:n//2]

plt.figure(figsize=(10, 6))
plt.plot(positive_frequencies, positive_power, label='Magnitude Spectrum')
plt.xlabel('Frequency')
plt.ylabel('Power')
plt.title('Frequency Spectrum of Filtered EMG Signal')
plt.grid(True)
plt.legend()
plt.show()